In [ ]:
CREATE SCHEMA Budget;

Commands completed successfully.

Total execution time: 00:00:00.088

In [ ]:
/*
===============================================================================
  TABLE:              [Budget].[Budget]
===============================================================================
  AUTHOR:             Sadia Sharmin
  CREATED:            2025-05-17
  VERSION:            1.0

  DESCRIPTION:
    Takes in columns from Reference.Budget,
    drops original BudgetDetail and BudgetElement columns for normalization.
    Created BudgetDetailID column to be a foreign key to connect to helper tables.
    Table has information on budget amounts and month/year of the budget

  COLUMNS:
    - [BudgetID]         : INT              - Primary key.
    - [BudgetValue]      : MONEY            - Money amount for budget.
    - [BudgetDetailID]   : INT              - Foreign Key
    - [Month]            : INT              - Month for budget
    - [Year]             : INT              - Year for budget

  CONSTRAINTS:
    - PRIMARY KEY: [BudgetID]
    - FOREIGN KEY: [BudgetDetailID]
    - DEFAULT:     [Year] defaults to 0000
                   [Month] default to 1
                   [BudgetValue] defaults to 0.00

  UDTs USED:
    - [SurrogateKeyInt]   for [BudgetID, BudgetDetailID]
    - [Year]              for [Year]
    - [Month]             for [Month]
    - [BudgetValue]       for [BudgetValue]

  MODIFICATION HISTORY:
  ------------------------------------------------------------------------------
  DATE        AUTHOR             DESCRIPTION
  ----------  -----------------  -----------------------------------------------
  2025-05-17  Sadia Sharmin      Initial creation with constraints
===============================================================================
*/

ALTER SCHEMA Budget TRANSFER Reference.Budget;

EXEC sp_rename 'Budget.Budget.BudgetKey', 'BudgetID', 'COLUMN';

DROP TYPE IF EXISTS Udt.Year;
CREATE TYPE Udt.Year FROM INT;

DROP TYPE IF EXISTS Udt.Month;
CREATE TYPE Udt.Month FROM INT;

DROP TYPE IF EXISTS Udt.BudgetValue;
CREATE TYPE Udt.BudgetValue FROM MONEY;

ALTER TABLE Budget.Budget
ALTER COLUMN BudgetID Udt.SurrogateKeyInt;

ALTER TABLE Budget.Budget
ALTER COLUMN BudgetDetailID Udt.SurrogateKeyInt;

ALTER TABLE Budget.Budget
DROP COLUMN BudgetDetail, BudgetElement;

ALTER TABLE Budget.Budget
ADD CONSTRAINT PK_Budget
PRIMARY KEY (BudgetID, BudgetDetailID);

ALTER TABLE Budget.Budget
ADD CONSTRAINT FK_Budget_BudgetDetail
FOREIGN KEY (BudgetDetailID) REFERENCES Budget.BudgetDetail(BudgetDetailID);

ALTER TABLE Budget.Budget
ALTER COLUMN Year Udt.Year;
ALTER TABLE Budget.Budget
ADD CONSTRAINT DF_Budget_Year DEFAULT 0000 FOR Year;

ALTER TABLE Budget.Budget
ALTER COLUMN Month Udt.Month;
ALTER TABLE Budget.Budget
ADD CONSTRAINT DF_Budget_Month DEFAULT 1 FOR Month;

ALTER TABLE Budget.Budget
ADD BudgetValue Udt.BudgetValue
CONSTRAINT DF_Budget_Value DEFAULT 0.00;


: Msg 15151, Level 16, State 1, Line 1
Cannot find the object 'Budget', because it does not exist or you do not have permission.

: Msg 15248, Level 11, State 1, Procedure sp_rename, Line 247
Either the parameter @objname is ambiguous or the claimed @objtype (COLUMN) is wrong.

: Msg 4924, Level 16, State 1, Line 11
ALTER TABLE DROP COLUMN failed because column 'BudgetDetail' does not exist in table 'Budget'.

Total execution time: 00:00:00.196

In [ ]:
/*
===============================================================================
  TABLE:              [Budget].[BudgetDetail]
===============================================================================
  AUTHOR:             Sadia Sharmin
  CREATED:            2025-05-17
  VERSION:            1.0

  DESCRIPTION:
    New table created as a helper to hold the BudgetDetail information.

  COLUMNS:
    - [BudgetDetailID]   : INT              - Primary key.
    - [BudgetDetail]     : DESCRIPTION      - Holds the details of the budget.
    - [BudgetCategoryID] : INT              - Foreign Key

  CONSTRAINTS:
    - PRIMARY KEY: [BudgetDetailID]
    - FOREIGN KEY: [BudgetCategoryID]

  UDTs USED:
    - [SurrogateKeyInt]   for [BudgetDetailID, BudgetCategoryID]
    - [Description]       for [BudgetDetail]

  MODIFICATION HISTORY:
  ------------------------------------------------------------------------------
  DATE        AUTHOR             DESCRIPTION
  ----------  -----------------  -----------------------------------------------
  2025-05-17  Sadia Sharmin      Initial creation with constraints
===============================================================================
*/

DROP TABLE IF EXISTS Budget.BudgetDetail;

DROP TYPE IF EXISTS Udt.Description
CREATE TYPE Udt.Description FROM NVARCHAR(20);

CREATE TABLE Budget.BudgetDetail(
    BudgetDetailID Udt.SurrogateKeyInt PRIMARY KEY,
    BudgetCategoryID Udt.SurrogateKeyInt NOT NULL,
    BudgetDetail Udt.Description NOT NULL,

    CONSTRAINT FK_BudgetDetail_BudgetCategory
        FOREIGN KEY (BudgetCategoryID) REFERENCES Budget.BudgetCategory(BudgetCategoryID)
);

: Msg 2715, Level 16, State 6, Line 6
Column, parameter, or variable #3: Cannot find data type Udt.Description.

Total execution time: 00:00:00.019

In [ ]:
/*
===============================================================================
  TABLE:              [Budget].[BudgetCategory]
===============================================================================
  AUTHOR:             Sadia Sharmin
  CREATED:            2025-05-17
  VERSION:            1.0

  DESCRIPTION:
    New table created as a helper to hold the BudgetCategory information.

  COLUMNS:
    - [BudgetCategoryID] : INT              - Primary key.
    - [BudgetCategory]   : DESCRIPTION      - Holds the category descriptions.

  CONSTRAINTS:
    - PRIMARY KEY: [BudgetCategoryID]

  UDTs USED:
    - [SurrogateKeyInt]   for [BudgetCategoryID]
    - [Description]       for [BudgetCategory]

  MODIFICATION HISTORY:
  ------------------------------------------------------------------------------
  DATE        AUTHOR             DESCRIPTION
  ----------  -----------------  -----------------------------------------------
  2025-05-17  Sadia Sharmin      Initial creation with constraints
===============================================================================
*/

DROP TABLE IF EXISTS Budget.BudgetCategory;

CREATE TABLE Budget.BudgetCategory (
    BudgetCategoryID Udt.SurrogateKeyInt PRIMARY KEY,
    BudgetCategory Udt.Description NOT NULL
);


In [ ]:
CREATE SCHEMA Employee;

Commands completed successfully.

Total execution time: 00:00:00.100

In [ ]:
/*
===============================================================================
  TABLE:              [Employee].[Staff]
===============================================================================
  AUTHOR:             Sadia Sharmin
  CREATED:            2025-05-17
  VERSION:            1.0

  DESCRIPTION:
    Takes in columns from Reference.Staff,
    holds information on the staff.
    ManagerID serves as a recursice foreign key that points to StaffID.
    DepartmentID references Employee.Department,
    which is a helper table with department names

  COLUMNS:
    - [StaffID ]         : INT              - Primary key.
    - [StaffName]        : Name             - Names of the staff.
    - [ManagerID]        : INT              - Foreign Key
    - [DepartmentID]     : INT              - Foreign Key

  CONSTRAINTS:
    - PRIMARY KEY: [STaffID]
    - FOREIGN KEY: [ManagerID, DepartmentID]
    - DEFAULT:     [StaffName] defaults to 'Unspecified'


  UDTs USED:
    - [SurrogateKeyInt]   for [StaffID, ManagerID, DepartmentID]
    - [Name]              for [StaffName]

  MODIFICATION HISTORY:
  ------------------------------------------------------------------------------
  DATE        AUTHOR             DESCRIPTION
  ----------  -----------------  -----------------------------------------------
  2025-05-17  Sadia Sharmin      Initial creation with constraints
===============================================================================
*/

ALTER SCHEMA Employee TRANSFER Reference.Staff;

DROP TYPE IF EXISTS Udt.Name;
CREATE TYPE Udt.Name FROM NVARCHAR(50);

ALTER TABLE Employee.Staff
ALTER COLUMN StaffID Udt.SurrogateKeyInt;

ALTER TABLE Employee.Staff
ALTER COLUMN DepartmentID Udt.SurrogateKeyInt;

ALTER TABLE Employee.Staff
ALTER COLUMN ManagerID Udt.SurrogateKeyInt;

ALTER TABLE Employee.Staff
ALTER COLUMN StaffName Udt.Name;

ALTER TABLE Employee.Staff
ADD CONSTRAINT DF_StaffName_Default DEFAULT 'UNSPECIFIED' FOR StaffName;

ALTER TABLE Employee.Staff
ADD CONSTRAINT Staff
PRIMARY KEY (StaffID);

ALTER TABLE Employee.Staff
ADD CONSTRAINT FK_Staff_Department
    FOREIGN KEY (DepartmentID) REFERENCES Employee.Department(DepartmentID),

    CONSTRAINT FK_Staff_Manager
    FOREIGN KEY (ManagerID) REFERENCES Employee.Staff(StaffID);



: Msg 2714, Level 16, State 5, Line 3
There is already an object named 'Staff' in the database.

: Msg 1750, Level 16, State 1, Line 3
Could not create constraint or index. See previous errors.

Total execution time: 00:00:00.299

In [ ]:
/*
===============================================================================
  TABLE:              [Employee].[Department]
===============================================================================
  AUTHOR:             Sadia Sharmin
  CREATED:            2025-05-17
  VERSION:            1.0

  DESCRIPTION:
    New table created as a helper to hold the department information.

  COLUMNS:
    - [DepartmentID]     : INT              - Primary key.
    - [Department]       : DESCRIPTION      - Holds the department descriptions.

  CONSTRAINTS:
    - PRIMARY KEY: [DepartmentID]

  UDTs USED:
    - [SurrogateKeyInt]   for [DepartmentID]
    - [Description]       for [Department]

  MODIFICATION HISTORY:
  ------------------------------------------------------------------------------
  DATE        AUTHOR             DESCRIPTION
  ----------  -----------------  -----------------------------------------------
  2025-05-17  Sadia Sharmin      Initial creation with constraints
===============================================================================
*/

DROP TABLE IF EXISTS Employee.Department

CREATE TABLE Employee.Department (
    DepartmentID Udt.SurrogateKeyInt PRIMARY KEY,
    Department Udt.Description NOT NULL
);

: Msg 2715, Level 16, State 6, Line 3
Column, parameter, or variable #2: Cannot find data type Udt.Description.

Total execution time: 00:00:00.025